# Multinomial Logistic Regression Model

To be run after data cleaning process

In [1]:
#Imports

import numpy as np
import pandas as pd
import warnings
import os
warnings.filterwarnings("ignore")

import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import norm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score as AUC, accuracy_score as Acc
from sklearn.model_selection import KFold, LeaveOneOut, cross_validate as CV
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression as LR
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import MNLogit
from itertools import chain, combinations

In [2]:
data = pd.read_csv('ml_preppred_surve.csv')
data.shape

(165, 17)

In [3]:
data = data.drop(columns = ['Unnamed: 0', 'Which is the main one?'])
data['Resident:  Permanent '] = data['Resident:  Permanent '].fillna(value = 0)
data = data.dropna()
data.shape

(131, 15)

In [4]:
data['Civil state'][(data['Civil state'] == 2) | (data['Civil state'] == 3)] = 100
data['Civil state'][data['Civil state'] != 100] = 200
data['Civil state'] = data['Civil state']/100
data.shape

(131, 15)

In [5]:
data['House'][data['House'] == 5] = 100
data['House'][data['House'] != 100] = 200
data['House'] = data['House']/100
data.shape

(131, 15)

In [6]:
data.head()

,Community name,Where do you normally cook?,What is the family's monthly income? (Brazil´s Minimum Wage R$937.00),Resident: Permanent,Gender (M or F),Age,Civil state,Education,House,How many meals a day?,. How many people eat at home daily?,How many contribute to the family's monthly income?,FuelChoice,Number_Men,Number_Women
2,1,3.0,2.0,1.0,2.0,3,2.0,3.0,1.0,4.0,2.0,2.0,2,1,1
3,1,3.0,2.0,1.0,1.0,2,1.0,4.0,1.0,3.0,4.0,1.0,2,2,2
5,1,3.0,3.0,1.0,2.0,1,1.0,3.0,1.0,3.0,3.0,1.0,4,1,2
6,1,3.0,3.0,1.0,1.0,1,2.0,3.0,1.0,3.0,4.0,1.0,4,2,2
7,1,3.0,3.0,1.0,1.0,4,2.0,6.0,1.0,4.0,4.0,2.0,4,3,1


In [7]:
y = data['FuelChoice']
# Switching categories 1 and 3, so that LPG- Charcoal is the reference category
y[y == 1] = -1
y[y == 3] = 1
y[y == -1] = 3
y.unique()

array([2, 4, 1, 3], dtype=int64)

In [8]:
data.columns

Index(['Community name', 'Where do you normally cook? ',
       'What is the family's monthly income? (Brazil´s Minimum Wage R$937.00)',
       'Resident:  Permanent ', 'Gender (M or F)', 'Age', 'Civil state',
       'Education', 'House', 'How many meals a day? ',
       '. How many people eat at home daily? ',
       'How many contribute to the family's monthly income? ', 'FuelChoice',
       'Number_Men', 'Number_Women'],
      dtype='object')

In [9]:
#X = data.drop(columns = ['FuelChoice', 'How many contribute to the family\'s monthly income? '])
X = data.drop(columns = ['FuelChoice'])
X = pd.get_dummies(X, columns = ['Community name', 'Where do you normally cook? ', 'Civil state', 'House'], drop_first = True)
X.head()

,What is the family's monthly income? (Brazil´s Minimum Wage R$937.00),Resident: Permanent,Gender (M or F),Age,Education,How many meals a day?,. How many people eat at home daily?,How many contribute to the family's monthly income?,Number_Men,Number_Women,...,Community name_8,Community name_9,Community name_10,Community name_11,Community name_12,Community name_13,Community name_14,Where do you normally cook? _3.0,Civil state_2.0,House_2.0
2,2.0,1.0,2.0,3,3.0,4.0,2.0,2.0,1,1,...,0,0,0,0,0,0,0,1,1,0
3,2.0,1.0,1.0,2,4.0,3.0,4.0,1.0,2,2,...,0,0,0,0,0,0,0,1,0,0
5,3.0,1.0,2.0,1,3.0,3.0,3.0,1.0,1,2,...,0,0,0,0,0,0,0,1,0,0
6,3.0,1.0,1.0,1,3.0,3.0,4.0,1.0,2,2,...,0,0,0,0,0,0,0,1,1,0
7,3.0,1.0,1.0,4,6.0,4.0,4.0,2.0,3,1,...,0,0,0,0,0,0,0,1,1,0


In [10]:
X.columns.values

array(["What is the family's monthly income? (Brazil´s Minimum Wage R$937.00)",
       'Resident:  Permanent ', 'Gender (M or F)', 'Age', 'Education',
       'How many meals a day? ', '. How many people eat at home daily? ',
       "How many contribute to the family's monthly income? ",
       'Number_Men', 'Number_Women', 'Community name_2',
       'Community name_3', 'Community name_4', 'Community name_5',
       'Community name_6', 'Community name_7', 'Community name_8',
       'Community name_9', 'Community name_10', 'Community name_11',
       'Community name_12', 'Community name_13', 'Community name_14',
       'Where do you normally cook? _3.0', 'Civil state_2.0', 'House_2.0'],
      dtype=object)

In [11]:
gammavals = np.arange(0, 0.5, 0.01)

kf = KFold(n_splits=5, random_state=42, shuffle=True)
loo = LeaveOneOut()
Xs = MinMaxScaler().fit_transform(X)
Xs = pd.DataFrame(Xs); Xs.columns = X.columns.values; Xs.index = X.index

Xc = sm.add_constant(Xs)
best_acc = -np.inf
for gamma in gammavals:
    print('gamma =', gamma)
        
    acc_list = []
    #for train_index, test_index in kf.split(Xc):   
        
    #    Xtrain, Xtest = Xc.iloc[train_index], Xc.iloc[test_index]
    #    ytrain, ytest = y.iloc[train_index], y.iloc[test_index]
        
    #    nunique = Xtrain.apply(pd.Series.nunique)
    #    cols_to_drop = nunique[nunique == 1].index
  
    #   Xtest = Xtest.drop(cols_to_drop, axis=1)
    #    Xtrain = Xtrain.drop(cols_to_drop, axis=1)
    
    #    MNLogitMdl = MNLogit(ytrain, Xtrain)
    #    results = MNLogitMdl.fit_regularized(method = 'l1', alpha = gamma, full_output = True, disp = False)

    #    ypred = np.argmax(results.predict(Xtest, linear = True).values, axis = 1) + 1
    #    acc_list.append(Acc(ytest.values, ypred))
    
    MNLogitMdl = MNLogit(y, Xc)
    results = MNLogitMdl.fit_regularized(method = 'l1', alpha = gamma, full_output = True, disp = False)
    ypred = np.argmax(results.predict(Xc, linear = True).values, axis = 1) + 1
    acc_list.append(Acc(y.values, ypred))
    
    acc = np.mean(acc_list)
    print('accuracy = ', acc)
    
    if acc > best_acc:
        best_acc = acc
        gamma_opt = gamma

print(gamma_opt, best_acc)

gamma = 0.0
accuracy =  0.6412213740458015
gamma = 0.01


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\l1_solvers_common.py:71: ConvergenceWarning: QC check did not pass for 3 out of 81 parameters
Try increasing solver accuracy or number of iterations, decreasing alpha, or switch solvers
  warnings.warn(message, ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\l1_solvers_common.py:144: ConvergenceWarning: Could not trim params automatically due to failed QC check. Trimming using trim_mode == 'size' will still work.
  warnings.warn(msg, ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


accuracy =  0.6412213740458015
gamma = 0.02
accuracy =  0.5038167938931297
gamma = 0.03
accuracy =  0.6183206106870229
gamma = 0.04
accuracy =  0.648854961832061
gamma = 0.05
accuracy =  0.6564885496183206
gamma = 0.06
accuracy =  0.6564885496183206
gamma = 0.07
accuracy =  0.6412213740458015
gamma = 0.08
accuracy =  0.6335877862595419
gamma = 0.09
accuracy =  0.6335877862595419
gamma = 0.1
accuracy =  0.6259541984732825
gamma = 0.11
accuracy =  0.6335877862595419
gamma = 0.12
accuracy =  0.6335877862595419
gamma = 0.13
accuracy =  0.6335877862595419
gamma = 0.14
accuracy =  0.6183206106870229
gamma = 0.15
accuracy =  0.6183206106870229
gamma = 0.16
accuracy =  0.6183206106870229
gamma = 0.17
accuracy =  0.6183206106870229
gamma = 0.18
accuracy =  0.6183206106870229
gamma = 0.19
accuracy =  0.6183206106870229
gamma = 0.2
accuracy =  0.6259541984732825
gamma = 0.21
accuracy =  0.6183206106870229
gamma = 0.22
accuracy =  0.6106870229007634
gamma = 0.23
accuracy =  0.6106870229007634
gamm

In [12]:
MNLogitMdl = MNLogit(y, Xc)
bestResults = MNLogitMdl.fit_regularized(method = 'l1', alpha = gamma_opt, full_output = True, disp = False)

In [13]:
MNLogitMdl = MNLogit(y, Xc)
baseResults = MNLogitMdl.fit_regularized(method = 'l1', alpha = 0, full_output = True, disp = False)

In [14]:
pvals = bestResults.pvalues.values
coefs = bestResults.params.values

table = np.column_stack(([coefs[1:,0], pvals[1:,0], coefs[1:,1], pvals[1:,1], coefs[1:,2], pvals[1:,2]]))
df = pd.DataFrame(table)
df.index = X.columns
df = df.dropna()
df.columns = ['weight_{G,C,F}', 'pval_{G,C,F}', 'weight_{G,F}', 'pval_{G,F}', 'weight_{G}', 'pval_{G}']
df

,"weight_{G,C,F}","pval_{G,C,F}","weight_{G,F}","pval_{G,F}",weight_{G},pval_{G}
What is the family's monthly income? (Brazil´s Minimum Wage R$937.00),-5.017123,0.054187,0.824192,0.737796,-1.580017,0.480422
Resident: Permanent,-2.332391,0.385247,-1.349157,0.604573,-1.919843,0.451629
Gender (M or F),-0.970940,0.251413,-0.566864,0.563024,-0.535406,0.539128
Age,-2.210070,0.126844,-0.843154,0.608636,-0.684768,0.648172
Education,-0.182376,0.926046,-1.246345,0.547374,0.954495,0.632468
How many meals a day?,1.495146,0.501187,2.482171,0.299470,2.007869,0.383923
. How many people eat at home daily?,5.988167,0.306820,11.113638,0.091443,4.019533,0.591104
How many contribute to the family's monthly income?,1.351928,0.668925,1.642034,0.625909,-1.127198,0.746505
Number_Women,-1.251296,0.609974,-2.307783,0.426275,-1.275258,0.657655
Community name_2,0.234539,0.882414,0.714718,0.757502,-1.030303,0.513402


In [15]:
##Added By Nandor
# Modify and print to LaTeX
display(df)
df_int = df.copy(deep=True)

change_name = {
"What is the family's monthly income? (Brazil´s Minimum Wage R$937.00)" : "Monthly Income of the Family",
 'Resident:  Permanent ' : "Residence",
 'Gender (M or F)': "Gender of Household Head",
 'Age' : "Age of Household Head",
 'Education': "Education of Household Head",
 'How many meals a day? ' : "Number of meals per day",
 '. How many people eat at home daily? ': "Number of people at meals daily",
 "How many contribute to the family's monthly income? ": "Number of people contributing to the monthly income",
 'Civil state_2.0':"Civil Status of Household Head",
 'House_2.0':"House Ownership"
}

#Community names from cleaning notebook
comm_name =  {'Aruaú': 1,
                    'Baixote': 2,
                    'Chita': 3,
                    'Marajá': 4,
                    'Monte Sinai': 5,
                    'Nova Esperança': 6,
                    'Nova Sugar caneã': 7,
                    'Pagodão': 8,
                    'Santa Isabel': 9,
                    'Santo Antônio': 10,
                    'São Tomé': 11,
                    'Terra Preta': 12,
                    'Tiririca': 13,
                    'Três unitos': 14}
#reverse_dict
comm_name = {comm_name[x]:x for x in comm_name}

#Modify For Print
df_int['Variable Name'] = df_int.index

for change in change_name:
    df_int.loc[df_int['Variable Name']==change,'Variable Name']= change_name[change]
    df_int.rename(index={change:change_name[change]},inplace=True)
    
for v_n in list(df_int['Variable Name'].values):
    if "Community name" in v_n:
        df_int.loc[df_int['Variable Name']==v_n,'Variable Name']= "qquad "+comm_name[int(v_n.split("_")[1])]
        df_int.rename(index={v_n:"Community name_"+comm_name[int(v_n.split("_")[1])]},inplace=True)
        
df_int.loc["Community name","Variable Name"]= "Community name"
df_int.sort_index(inplace=True)

df_int = df_int[["Variable Name","weight_{G,C,F}","pval_{G,C,F}","weight_{G,F}","pval_{G,F}","weight_{G}","pval_{G}"]]
display(df_int)
df_int.to_csv('table6.csv')

def raw(x):
    return x

def format_w(x):
    if np.isnan(x):
        return ""
    else:
        return '%.2f' % x

def format_p(x):
    if np.isnan(x):
        return ""
    else:
        return '%.4f' % x

with pd.option_context("max_colwidth", 1000):
    string_tex = df_int.to_latex(header = False, formatters=[raw,format_w,format_p,format_w,format_p,format_w,format_p],multirow=True,index=False)

add_header = """ 
\\begin{tabular}{|p{5.5cm}|p{1.2cm}p{1.2cm}|p{1.2cm}p{1.2cm}|p{1.2cm}p{1.2cm}|}
                \\toprule
				&\\multicolumn{2}{c|}{G, C, F}&\\multicolumn{2}{c|}{G, F}&\\multicolumn{2}{c|}{G}\\\\
				\\cline{2-3}\\cline{4-5}\\cline{6-7}
				
				Variable name & $w_4$ & p-value& $w_5$ & p-value & $w_6$ & p-value \\\\
				\midrule
"""
string_tex = string_tex.replace("qquad","\\qquad")

string_tex = add_header + string_tex.split("toprule")[1]

with open("table6_format.tex", "wb") as text_file: 
    text_file.write(string_tex.encode('utf8')) 


,"weight_{G,C,F}","pval_{G,C,F}","weight_{G,F}","pval_{G,F}",weight_{G},pval_{G}
What is the family's monthly income? (Brazil´s Minimum Wage R$937.00),-5.017123,0.054187,0.824192,0.737796,-1.580017,0.480422
Resident: Permanent,-2.332391,0.385247,-1.349157,0.604573,-1.919843,0.451629
Gender (M or F),-0.970940,0.251413,-0.566864,0.563024,-0.535406,0.539128
Age,-2.210070,0.126844,-0.843154,0.608636,-0.684768,0.648172
Education,-0.182376,0.926046,-1.246345,0.547374,0.954495,0.632468
How many meals a day?,1.495146,0.501187,2.482171,0.299470,2.007869,0.383923
. How many people eat at home daily?,5.988167,0.306820,11.113638,0.091443,4.019533,0.591104
How many contribute to the family's monthly income?,1.351928,0.668925,1.642034,0.625909,-1.127198,0.746505
Number_Women,-1.251296,0.609974,-2.307783,0.426275,-1.275258,0.657655
Community name_2,0.234539,0.882414,0.714718,0.757502,-1.030303,0.513402


,Variable Name,"weight_{G,C,F}","pval_{G,C,F}","weight_{G,F}","pval_{G,F}",weight_{G},pval_{G}
Age of Household Head,Age of Household Head,-2.210070,0.126844,-0.843154,0.608636,-0.684768,0.648172
Civil Status of Household Head,Civil Status of Household Head,0.458184,0.599620,0.842147,0.404873,1.171706,0.212815
Community name,Community name,NaN,NaN,NaN,NaN,NaN,NaN
Community name_Baixote,qquad Baixote,0.234539,0.882414,0.714718,0.757502,-1.030303,0.513402
Community name_Marajá,qquad Marajá,2.946463,0.315701,4.679522,0.148687,2.630078,0.360710
Community name_Monte Sinai,qquad Monte Sinai,1.662134,0.558376,3.502040,0.275796,1.656405,0.552309
Community name_Nova Esperança,qquad Nova Esperança,-0.522502,0.696872,-0.272707,0.899245,-0.996211,0.440913
Community name_Nova Sugar caneã,qquad Nova Sugar caneã,-0.007555,0.998876,4.885425,0.352434,0.922914,0.859007
Community name_Pagodão,qquad Pagodão,-0.650829,0.651099,1.658528,0.401105,-1.687560,0.268551
Community name_Santo Antônio,qquad Santo Antônio,1.724819,0.328825,2.325196,0.334460,-0.357338,0.838714


In [16]:
df.shape

(20, 6)

In [17]:
pvals = baseResults.pvalues.values
coefs = baseResults.params.values

table = np.column_stack(([coefs[1:,0], pvals[1:,0], coefs[1:,1], pvals[1:,1], coefs[1:,2], pvals[1:,2]]))
df_base = pd.DataFrame(table)
df_base.index = X.columns
#df_base = df_base.dropna()
df_base.columns = ['weight_{G,C,F}', 'pval_{G,C,F}', 'weight_{G,F}', 'pval_{G,F}', 'weight_{G}', 'pval_{G}']
df_base

,"weight_{G,C,F}","pval_{G,C,F}","weight_{G,F}","pval_{G,F}",weight_{G},pval_{G}
What is the family's monthly income? (Brazil´s Minimum Wage R$937.00),-5.075563,0.090042,1.486227,0.599860,-1.428691,0.573171
Resident: Permanent,-12.657528,0.957208,-11.536068,0.960997,-11.998912,0.959432
Gender (M or F),-1.242361,0.222715,-0.862858,0.453265,-0.812542,0.429123
Age,-4.619220,0.023717,-3.386651,0.133429,-2.953299,0.153426
Education,-2.244878,0.379449,-3.406284,0.201123,-0.935312,0.715788
How many meals a day?,2.174695,0.426737,3.441667,0.238143,2.625428,0.346022
. How many people eat at home daily?,22.508330,0.095501,28.885913,0.036423,22.028540,0.116644
How many contribute to the family's monthly income?,4.236916,0.311567,4.590322,0.288693,1.479929,0.741942
Number_Men,-4.766971,0.337121,-5.312839,0.298972,-6.183432,0.236448
Number_Women,-6.130529,0.115519,-7.788979,0.069035,-6.614422,0.120548


In [18]:
df_base.shape

(26, 6)

# Reference: Gas and Charcoal

In [20]:
##Added By Nandor
# Modify and print to LaTeX
display(df_base)
df_int = df_base.copy(deep=True)

change_name = {
"What is the family's monthly income? (Brazil´s Minimum Wage R$937.00)" : "Monthly Income of the Family",
 'Resident:  Permanent ' : "Residence",
 'Gender (M or F)': "Gender of Household Head",
 'Age' : "Age of Household Head",
 'Education': "Education of Household Head",
  "Number_Men":"Number of Men in Household",
  "Number_Women":"Number of Women in Household",
 'How many meals a day? ' : "Number of meals per day",
 '. How many people eat at home daily? ': "Number of people at meals daily",
 "How many contribute to the family's monthly income? ": "Number of people contributing to the monthly income",
 'Civil state_2.0':"Civil Status of Household Head",
 'House_2.0':"House Ownership",
  "Where do you normally cook? _3.0":"Normal Cooking Area"
}

#Community names from cleaning notebook
comm_name =  {'Aruaú': 1,
                    'Baixote': 2,
                    'Chita': 3,
                    'Marajá': 4,
                    'Monte Sinai': 5,
                    'Nova Esperança': 6,
                    'Nova Sugar caneã': 7,
                    'Pagodão': 8,
                    'Santa Isabel': 9,
                    'Santo Antônio': 10,
                    'São Tomé': 11,
                    'Terra Preta': 12,
                    'Tiririca': 13,
                    'Três unitos': 14}
#reverse_dict
comm_name = {comm_name[x]:x for x in comm_name}

#Modify For Print
df_int['Variable Name'] = df_int.index

for change in change_name:
    df_int.loc[df_int['Variable Name']==change,'Variable Name']= change_name[change]
    df_int.rename(index={change:change_name[change]},inplace=True)
    
for v_n in list(df_int['Variable Name'].values):
    if "Community name" in v_n:
        df_int.loc[df_int['Variable Name']==v_n,'Variable Name']= "qquad "+comm_name[int(v_n.split("_")[1])]
        df_int.rename(index={v_n:"Community name_"+comm_name[int(v_n.split("_")[1])]},inplace=True)
        
df_int.loc["Community name","Variable Name"]= "Community name"
df_int.sort_index(inplace=True)

df_int = df_int[["Variable Name","weight_{G,C,F}","pval_{G,C,F}","weight_{G,F}","pval_{G,F}","weight_{G}","pval_{G}"]]
display(df_int)
df_int.to_csv('table6.csv')

def raw(x):
    return x

def format_w(x):
    if np.isnan(x):
        return ""
    else:
        return '%.2f' % x

def format_p(x):
    if np.isnan(x):
        return ""
    else:
        return '%.4f' % x

with pd.option_context("max_colwidth", 1000):
    string_tex = df_int.to_latex(header = False, formatters=[raw,format_w,format_p,format_w,format_p,format_w,format_p],multirow=True,index=False)

add_header = """ 
\\begin{tabular}{|p{5.5cm}|p{1.2cm}p{1.2cm}|p{1.2cm}p{1.2cm}|p{1.2cm}p{1.2cm}|}
                \\toprule
				&\\multicolumn{2}{c|}{G, C, F}&\\multicolumn{2}{c|}{G, F}&\\multicolumn{2}{c|}{G}\\\\
				\\cline{2-3}\\cline{4-5}\\cline{6-7}
				
				Variable name & $w_4$ & p-value& $w_5$ & p-value & $w_6$ & p-value \\\\
				\midrule
"""
string_tex = string_tex.replace("qquad","\\qquad")

string_tex = add_header + string_tex.split("toprule")[1]

with open("table8_format.tex", "wb") as text_file: 
    text_file.write(string_tex.encode('utf8')) 


,"weight_{G,C,F}","pval_{G,C,F}","weight_{G,F}","pval_{G,F}",weight_{G},pval_{G}
What is the family's monthly income? (Brazil´s Minimum Wage R$937.00),-5.075563,0.090042,1.486227,0.599860,-1.428691,0.573171
Resident: Permanent,-12.657528,0.957208,-11.536068,0.960997,-11.998912,0.959432
Gender (M or F),-1.242361,0.222715,-0.862858,0.453265,-0.812542,0.429123
Age,-4.619220,0.023717,-3.386651,0.133429,-2.953299,0.153426
Education,-2.244878,0.379449,-3.406284,0.201123,-0.935312,0.715788
How many meals a day?,2.174695,0.426737,3.441667,0.238143,2.625428,0.346022
. How many people eat at home daily?,22.508330,0.095501,28.885913,0.036423,22.028540,0.116644
How many contribute to the family's monthly income?,4.236916,0.311567,4.590322,0.288693,1.479929,0.741942
Number_Men,-4.766971,0.337121,-5.312839,0.298972,-6.183432,0.236448
Number_Women,-6.130529,0.115519,-7.788979,0.069035,-6.614422,0.120548


,Variable Name,"weight_{G,C,F}","pval_{G,C,F}","weight_{G,F}","pval_{G,F}",weight_{G},pval_{G}
Age of Household Head,Age of Household Head,-4.619220,0.023717,-3.386651,0.133429,-2.953299,0.153426
Civil Status of Household Head,Civil Status of Household Head,1.162254,0.292896,1.649915,0.183498,1.964613,0.090317
Community name,Community name,NaN,NaN,NaN,NaN,NaN,NaN
Community name_Baixote,qquad Baixote,0.969264,0.589303,8.552948,0.879424,-0.456938,0.801879
Community name_Chita,qquad Chita,13.222499,0.960351,19.820894,0.941883,10.109015,0.969682
Community name_Marajá,qquad Marajá,13.504829,0.937191,22.289284,0.901669,13.025750,0.939414
Community name_Monte Sinai,qquad Monte Sinai,9.956167,0.924547,18.839594,0.874494,9.811392,0.925640
Community name_Nova Esperança,qquad Nova Esperança,-0.219286,0.887989,6.778622,0.904290,-0.760155,0.635875
Community name_Nova Sugar caneã,qquad Nova Sugar caneã,4.528651,0.998635,16.766535,0.994947,5.673387,0.998290
Community name_Pagodão,qquad Pagodão,-0.045175,0.977006,9.339442,0.868394,-1.264585,0.456192
